# **Efficient Yelp API Calls (Core)**

For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.



## **Install and import libraries**

In [1]:
# !pip install yelpapi
!pip install tqdm

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

 ## **Perform test Yelp API call**

In [3]:
# Load API Credentials
with open('C:/Users/DELL/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI instance
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [5]:
# Test API call
results = yelp_api.search_query(location='Austin, TX', term='pizza')

In [6]:
# Display the datatype for the results of the API call
type(results)

dict

In [7]:
# Display the keys for the disctionary
results.keys()

dict_keys(['businesses', 'total', 'region'])

## **Perform API Call**

### **Define Search Terms and File Paths**
- Choose the location and search term (food), but it must return more than 20 results so you can practice pagination.

In [8]:
# Set our API call parameters
LOCATION = "Austin, TX 78729"
TERM = "vietnamese"

In [9]:
# Specify folder for saving data
FOLDER = "Data/"
os.makedirs(FOLDER , exist_ok=True)

In [10]:
# Obtain the city from the location
CITY = LOCATION.split(',')[0]

'Austin'

In [11]:
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = f"{FOLDER}{TERM}_{CITY}.json"
JSON_FILE

'Data/vietnamese_Austin.json'

### **Check if JSON file exists and create it if it doesn't**

In [12]:
# Display the directory name of the JSON file
os.path.dirname(JSON_FILE)

'Data'

In [13]:
# Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: 
if file_exists == False:
    # Create any needed folders
    # Get the folder name
    folder = os.path.dirname(JSON_FILE)
    # If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)   
    # Inform the user
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    # saving empty list
    with open(JSON_FILE,'w') as f:
        json.dump([], f)
# Advise if the file already exists
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/vietnamese_Austin.json not found. Saving empty list to file.


### **Load JSON FIle and account for previous results**

In [14]:
# Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

# Display the number of previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### **Make a test API call to obtain the first page of data**

In [15]:
# Use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(term=TERM, location=LOCATION)

In [16]:
# Display the keys from the API call
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [17]:
# Display the total number of results
results['total']

96

In [18]:
# Display the businesses datatype
type(results['businesses'])

list

In [19]:
# Calculate the number of results per page 
results_per_page = len(results['businesses'])
# Display the number of results per page  
results_per_page

20

In [20]:
# Calculate how many pages of results needed to cover the total_results
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(results['total'] / results_per_page)
n_pages

5

### **Create a JSON file from the results of API calls**

In [21]:
# Loop through pages and append API call results to the JSON file
for i in tqdm_notebook(range(0, n_pages+1)):
    try:
        time.sleep(.2)
        # Load JSON file
        with open(JSON_FILE,'r') as f:
            previous_results = json.load(f)
        # Obtain current number of results in JSON file
        n_results = len(previous_results)
        # Obtain next page of results
        results = yelp_api.search_query(term=TERM, 
                                        location=LOCATION,
                                        offset=n_results+1)
        # Append this page of results and save to file
        previous_results.extend( results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(previous_results, f)
    # If there is an error exception, print the error message and break
    except Exception as e:
        print(e)
        break

  0%|          | 0/6 [00:00<?, ?it/s]

## **Save results as JSON and CSV files**

In [22]:
# Creat a dataframe for the JSON file
df = pd.read_json(JSON_FILE)
# View head and tail of dataframe
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,dv2xeItylQRjUEtxNsKBYw,pho-hoang-austin-5,Pho Hoang,https://s3-media2.fl.yelpcdn.com/bphoto/AWlijk...,False,https://www.yelp.com/biz/pho-hoang-austin-5?ad...,295,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 30.4791179, 'longitude': -97.6782...","[delivery, pickup]",$$,"{'address1': '399 W Louis Henna Blvd', 'addres...",+15122388008,(512) 238-8008,8004.906445
1,PgcB9iS3YFOZ1Evm3ResAw,pho-thanh-nhi-cedar-park-4,Pho Thanh Nhi,https://s3-media4.fl.yelpcdn.com/bphoto/HSaU0C...,False,https://www.yelp.com/biz/pho-thanh-nhi-cedar-p...,246,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 30.5299005202406, 'longitude': -9...",[delivery],$$,"{'address1': '1335 E Whitestone Blvd', 'addres...",+15122606363,(512) 260-6363,9855.999080
2,kV7bogiloOEgYSr_bU5VLw,sunflower-vietnamese-cuisine-austin,Sunflower Vietnamese Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/niGZq4...,False,https://www.yelp.com/biz/sunflower-vietnamese-...,65,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 30.44159719662341, 'longitude': -...",[],$$,"{'address1': '6001 W Parmer Ln', 'address2': '...",+15122916749,(512) 291-6749,1973.028364
3,3yv-AArdULfZmXiEWN5FHw,pho-phi-austin-3,Pho Phi,https://s3-media2.fl.yelpcdn.com/bphoto/THD99V...,False,https://www.yelp.com/biz/pho-phi-austin-3?adju...,351,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 30.41432985890226, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1700 W Parmer Ln', 'address2': '...",+15125962746,(512) 596-2746,8032.840120
4,Ck14Bt1fEY2UYZykyyJQ3Q,banh-mi-galang-austin,Banh Mi Galang,https://s3-media2.fl.yelpcdn.com/bphoto/nFUnEp...,False,https://www.yelp.com/biz/banh-mi-galang-austin...,62,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 30.479747760604138, 'longitude': ...","[delivery, pickup]",NaN,"{'address1': '11301 Lakeline Blvd', 'address2'...",+15125848391,(512) 584-8391,4853.009782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,LKoYVqQyWJbIaC2UkDMynw,sichuan-garden-round-rock,Sichuan Garden,https://s3-media3.fl.yelpcdn.com/bphoto/7SKII8...,False,https://www.yelp.com/biz/sichuan-garden-round-...,268,"[{'alias': 'szechuan', 'title': 'Szechuan'}, {...",3.5,"{'latitude': 30.5084141256793, 'longitude': -9...",[delivery],$$,"{'address1': '110 I H 35 N', 'address2': 'Ste ...",+15122556952,(512) 255-6952,8857.117003
91,jWz0NBCz5Ya2OSyeRNbI1w,masala-wok-austin-2,Masala Wok,https://s3-media3.fl.yelpcdn.com/bphoto/eliiIU...,False,https://www.yelp.com/biz/masala-wok-austin-2?a...,357,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,"{'latitude': 30.388905158257543, 'longitude': ...","[delivery, pickup]",$$,"{'address1': '10515 N Mopac Expy', 'address2':...",+15122962977,(512) 296-2977,7830.367277
92,VJj_xjjWX_UHaFNL91_Duw,masala-wok-austin,Masala Wok,https://s3-media2.fl.yelpcdn.com/bphoto/p21fkZ...,False,https://www.yelp.com/biz/masala-wok-austin?adj...,315,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,"{'latitude': 30.41804, 'longitude': -97.67123}","[delivery, pickup]",$,"{'address1': '1100 Center Ridge Dr', 'address2...",+15122519696,(512) 251-9696,9222.945104
93,hrDUqL02tA-6lnnopguEyg,teapioca-lounge-austin-5,Teapioca Lounge,https://s3-media1.fl.yelpcdn.com/bphoto/0WkTIn...,False,https://www.yelp.com/biz/teapioca-lounge-austi...,89,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,"{'latitude': 30.391071, 'longitude': -97.7487887}",[delivery],$,"{'address1': '10000 Research Blvd', 'address2'...",+15128383656,(512) 838-3656,7273.501896


In [23]:
df.duplicated(subset='id').sum()

0

In [24]:
# Convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
# Display the file name
csv_file

'Data/vietnamese_Austin.csv.gz'

In [25]:
# Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression='gzip', index=False)

### **Compare JSON and CSV_GZ filesizes**

In [26]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(csv_file)

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')
print('\n')
print(f'The csv.gz is {size_json/size_csv_gz:.2f} times smaller!')

JSON FILE: 93,601 Bytes
CSV.GZ FILE: 13,975 Bytes
the csv.gz is 6.70 times smaller!
